In [21]:
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.utils import indexable, _safe_indexing
from sklearn.utils.validation import _num_samples
from sklearn.model_selection._split import _validate_shuffle_split
from itertools import chain


In [22]:
def multilabel_train_test_split(*arrays,
                                test_size=None,
                                train_size=None,
                                random_state=None,
                                shuffle=True,
                                stratify=None):
    """
    Train test split for multilabel classification. Uses the algorithm from: 
    'Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of Multi-Label Data'.
    """
    if stratify is None:
        return train_test_split(*arrays, test_size=test_size,train_size=train_size,
                                random_state=random_state, stratify=None, shuffle=shuffle)
    
    assert shuffle, "Stratified train/test split is not implemented for shuffle=False"
    
    n_arrays = len(arrays)
    arrays = indexable(*arrays)
    n_samples = _num_samples(arrays[0])
    n_train, n_test = _validate_shuffle_split(
        n_samples, test_size, train_size, default_test_size=0.25
    )
    cv = MultilabelStratifiedShuffleSplit(test_size=n_test, train_size=n_train, random_state=123)
    train, test = next(cv.split(X=arrays[0], y=stratify))

    return list(
        chain.from_iterable(
            (_safe_indexing(a, train), _safe_indexing(a, test)) for a in arrays
        )
    )

In [23]:
data = pd.read_csv(r"DataFrameTest.csv")
x = data[['Subject']]
y = data[['Protocol/Folder','Age']]

subjects_train, subjects_test, age_group_train, age_group_test = multilabel_train_test_split(x,y,stratify=y, test_size=0.20)


In [24]:
strat_train = pd.concat([subjects_train, age_group_train], axis=1)
strat_test = pd.concat([subjects_test, age_group_test], axis=1)

strat_train = strat_train.reset_index(drop = True)
strat_test = strat_test.reset_index(drop = True)

In [25]:
with pd.ExcelWriter('stratify_3.xlsx') as writer:
    train.to_excel(writer, sheet_name='Train')
    strat_test.to_excel(writer, sheet_name='Test')